In [ ]:
from model_tbyt_3 import GPT, GPTConfig
from checkpoint_translator import load_checkpoint_to_gpt
checkpoint_path = 'Grid_training_without_duplicates/Final_N64_K16_L2_H1_E16_r4over1_npos1_mlp1_dup0_testK16_iters60000.pt'
#'Grid_training_without_duplicates/Final_N128_K16_L2_H1_E64_r2over1_npos0_mlp1_dup0_testK16_iters60000.pt'
model, checkpoint = load_checkpoint_to_gpt(
    checkpoint_path, 
    GPT, 
    GPTConfig,
    device='cpu',
    strict=False
)

KeyError: 'model_config'

In [1]:
from model_tbyt_3 import GPT, GPTConfig
import torch
import os
itr_num = 60000
block_size = 32
vocab_size = 128
device = 'cpu'
config = GPTConfig(block_size=block_size, vocab_size=vocab_size)
config.without_pos = True
model = GPT(config)
model_state_dict = torch.load(os.path.join(os.getcwd(), f'saved_models/jan27-tbyt_without-pos-embedding_n_embd:64_head:1_layers:2_vocab_size:128_itr:60000_checkpoint_time_1769551758.5343654.pt'), map_location=device)['model']
model.load_state_dict(model_state_dict)
model.to(device=device)

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(129, 64)
    (wpe): Embedding(129, 64)
    (h): ModuleList(
      (0-1): 2 x Block(
        (c_attn): CasualSelfAttention(
          (c_attn): Linear(in_features=64, out_features=192, bias=True)
          (c_proj): Linear(in_features=64, out_features=64, bias=True)
        )
        (c_fc): MLP(
          (fc_1): Linear(in_features=64, out_features=192, bias=True)
          (gelu): GELU(approximate='tanh')
          (fc_2): Linear(in_features=192, out_features=64, bias=True)
        )
        (ln_1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (ln_2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      )
    )
    (ln_f): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=64, out_features=128, bias=False)
)

In [2]:
batch_size = 1
# Use the loaded model's config instead of class defaults
vocab_size = model.config.vocab_size
block_size = model.config.block_size
#test_1_seq = torch.cat((torch.arange(110, 30, -5) , torch.arange(127, 111, -1)), dim=0) 
#print(test_1_seq)
def get_batch(changing_num=-1, changing_index=-1, initial_sequence=None, batch_size=batch_size):
   def cat_sorted_tensor(x):
      if initial_sequence is not None:
         x = initial_sequence
      else:
         x = x
         #x, _ = torch.sort(x, descending=True)
      if changing_num != -1:
         if changing_index == -1:
            x[0] = changing_num
         else:
            x[changing_index] = changing_num
      #x = torch.cat((torch.tensor([100]).repeat(16), torch.tensor([1]).repeat(16)))
      #x = torch.tensor([100,100,100,100,1,1,1,1])
      vals, _ = torch.sort(x)
      #vals2, _ = torch.sort(x, descending=True)
      #print('vals are ', vals)
      return torch.cat((x, torch.tensor([vocab_size]), vals), dim=0)
   #x = torch.stack([cat_sorted_tensor(torch.randperm(vocab_size)[:block_size]) for _ in range(batch_size)])
   x = torch.stack([cat_sorted_tensor(torch.randperm(vocab_size)[:block_size]) for _ in range(batch_size)])
   return x

In [3]:
import torch
idx = get_batch()
print('idx dim is ', idx.shape)
logits, loss = model(idx)
print('loss is ', loss.item())
print(f'idx is: {idx}')
print('model output is ', torch.argmax(logits, dim=-1))

idx dim is  torch.Size([1, 65])
layer_n is  0
layer_n is  0
layer_n is  1
layer_n is  1
loss is  0.0006695028860121965
idx is: tensor([[  2,  76, 120,  68, 107,  95,  17,  47,  73, 123,  38,   6, 116,  20,
           5,  12,  13,  67,  99,  39, 102,  58,  45,  97,  87,  85,  29, 106,
          61,  21,  26,  42, 128,   2,   5,   6,  12,  13,  17,  20,  21,  26,
          29,  38,  39,  42,  45,  47,  58,  61,  67,  68,  73,  76,  85,  87,
          95,  97,  99, 102, 106, 107, 116, 120, 123]])
model output is  tensor([[ 22,  97, 120,  76, 120, 107,  35,  66,  76, 123,  47,  17, 120,  38,
           6,  17,  17,  68, 107,  47, 107,  67,  47,  99,  95,  87,  38, 107,
          67,  29,  29,  45,   2,   5,   6,  12,  13,  17,  20,  21,  26,  29,
          38,  39,  42,  45,  47,  58,  61,  67,  68,  73,  76,  85,  87,  95,
          97,  99, 102, 106, 107, 116, 120, 123, 123]])
